##### ▶ライブラリ

In [2]:
import pandas as pd
import numpy as np
import requests
import io
import math
import copy
from scipy.stats import gmean
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import patches
from pipeline_functions import PipelineFunctions
pf = PipelineFunctions()
from view_functions import ViewFunctions
vf = ViewFunctions()

import umap
from sklearn.manifold import TSNE
import seaborn as sns
palette = ['#CC521D', '#4F4AD7', '#39AE3D']

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

##### ▶SVMを検証する

In [49]:
from sklearn.svm import SVC

### 変えないパラメータ ###
TRAIN_TEST_SPLIT_SEED = 1
KFOLD_SHUFFLE_SEED = 1
KFOLD_SHUFFLE_LABEL = 'saito_label'
Y_COL = 'is_good_saito'
K = 5

### 変えるパラメータ ###
PARAM_COMBO = pd.read_csv('../temp/param_combo.csv', delimiter=',', index_col=0)
PARAM_COMBO_GRAD = pd.read_csv('../temp/param_combo_grad.csv', delimiter=',', index_col=0)

### ハイパーパラメータ ###
SVM_PARAMETER = pd.read_csv('../temp/svm_parameter.csv', delimiter=',', index_col=0)

for i, row in PARAM_COMBO.iterrows():
    ### パラメータの組み合わせ ###
    ptcnt_type, ptcnt, input_col = row['ptcnt_type'], row['ptcnt'], row['X_col']
    
    ### 結果格納用df ###
    param_result = pd.read_csv('../temp/svm_parameter.csv', delimiter=',', index_col=0)
    param_result_dict = {}
    for col in param_result:
        param_result_dict[col] = param_result[col]
    param_result_dict['test_recall'] = []
    param_result_dict['test_precision'] = []
    param_result_dict['test_f1'] = []
    param_result_dict['test_accuracy'] = []

    data = pd.read_csv(f'../temp/eachpt_feature/{ptcnt_type}_{ptcnt}_{input_col}.csv', delimiter=',', index_col=0)
    X = data.copy()
    y = data[Y_COL]
    # 学習データとテストデータを725:310に分割する
    train_valid, test, y_train_valid, y_test = train_test_split(X, y, train_size=725, shuffle=True, stratify=y, random_state=TRAIN_TEST_SPLIT_SEED)
   
    for j, row in SVM_PARAMETER.iterrows():
        kernel, c, gamma, degree = row['kernel'], row['c'], row['gamma'], row['degree']
    
        # K-Fold
        skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=KFOLD_SHUFFLE_SEED)
        for k, (train_index, valid_index) in enumerate(skf.split(train_valid, train_valid[KFOLD_SHUFFLE_LABEL])):
            
            ### 訓練データと検証データに分ける ###
            train, valid = train_valid.iloc[train_index], train_valid.iloc[valid_index]
            
            ### 説明変数と目的変数に分ける ###
            X_col = [col for col in train_valid if 'feature_' in col]
            y_col = 'is_good_saito'
            train_X = train.copy()[X_col]
            train_y = train.copy()[y_col]
            valid_X = valid.copy()[X_col]
            valid_y = valid.copy()[y_col]
            
            ### 学習データのスケーリング(正規化, 学習データのmin, maxを検証データとテストデータに適用) ###
            scaler = MinMaxScaler()
            for col in train_X:
                train_minmax = scaler.fit(train_X[[col]])
                train_X[f'norm_{col}'] = scaler.transform(train_X[[col]])
                valid_X[f'norm_{col}'] = scaler.transform(valid_X[[col]])
                del train_X[col]
                del valid_X[col]
        
            ### モデルへの入力形式に変換する ###
            train_X = train_X.to_numpy()
            train_y = train_y.to_numpy()
            valid_X = valid_X.to_numpy()
            valid_y = valid_y.to_numpy()
        
            ### モデルを生成する ###
            model = SVC(kernel=kernel, C=c, gamma=gamma, degree=degree)
            ### モデルを学習させる ###
            result = model.fit(train_X, train_y)
            ### 検証データで精度を算出する ###
            valid_pred = model.predict(valid_X)
            ### 各指標の値(validationデータに対する) ###
            recall = recall_score(valid_y, valid_pred, average=None)[1]
            precision = precision_score(valid_y, valid_pred, average=None)[1]
            f1 = f1_score(valid_y, valid_pred, average=None)[1]
            accuracy = accuracy_score(valid_y, valid_pred)
            
            ### 結果を格納する ###
            param_result_dict[f'kfold_{k}_recall'].append(recall)
            param_result_dict[f'kfold_{k}_precision'].append(precision)
            param_result_dict[f'kfold_{k}_f1'].append(f1)
            param_result_dict[f'kfold_{k}_accuracy'].append(accuracy)
        
        ### 説明変数と目的変数に分ける ###
        X_col = [col for col in train_valid if 'feature_' in col]
        y_col = 'is_good_saito'
        train_valid_X = train_valid.copy()[X_col]
        train_valid_y = train_valid.copy()[y_col]
        test_X = test.copy()[X_col]
        test_y = test.copy()[y_col]

        ### 学習データのスケーリング(正規化, 学習データのmin, maxを検証データとテストデータに適用) ###
        scaler = MinMaxScaler()
        for col in train_valid_X:
            train_minmax = scaler.fit(train_valid_X[[col]])
            train_valid_X[f'norm_{col}'] = scaler.transform(train_valid_X[[col]])
            test_X[f'norm_{col}'] = scaler.transform(test_X[[col]])
            del train_valid_X[col]
            del test_X[col]
        
        ### モデルへの入力形式に変換する ###
        train_valid_X = train_valid_X.to_numpy()
        train_valid_y = train_valid_y.to_numpy()
        test_X = test_X.to_numpy()
        test_y = test_y.to_numpy()

        ### モデルを生成する ###
        model = SVC(kernel=kernel, C=c, gamma=gamma, degree=degree)
        ### モデルを学習させる ###
        result = model.fit(train_valid_X, train_valid_y)
        ### 検証データで精度を算出する ###
        test_pred = model.predict(test_X)
        ### 各指標の値(testデータに対する) ###
        recall = recall_score(test_y, test_pred, average=None)[1]
        precision = precision_score(test_y, test_pred, average=None)[1]
        f1 = f1_score(test_y, test_pred, average=None)[1]
        accuracy = accuracy_score(test_y, test_pred)
        
        ### 結果を格納する ###
        param_result_dict['test_recall'].append(recall)
        param_result_dict['test_precision'].append(precision)
        param_result_dict['test_f1'].append(f1)
        param_result_dict['test_accuracy'].append(accuracy)        
    
    param_result_aftertrain = pd.DataFrame(param_result_dict)
    param_result_aftertrain.to_csv(f'../temp/result_svm/{ptcnt_type}_{ptcnt}_{input_col}.csv')
    print(ptcnt_type, ptcnt)
    print('==================================')

c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\i

1 3


c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\i

1 3


c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\i

...

2 82
2 82
2 82


c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\i

2 83


c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\i

2 83
2 83
2 83


In [50]:
### 変えないパラメータ ###
TRAIN_TEST_SPLIT_SEED = 1
KFOLD_SHUFFLE_SEED = 1
KFOLD_SHUFFLE_LABEL = 'saito_label'
Y_COL = 'is_good_saito'
K = 5

### 変えるパラメータ ###
PARAM_COMBO = pd.read_csv('../temp/param_combo.csv', delimiter=',', index_col=0)
PARAM_COMBO_GRAD = pd.read_csv('../temp/param_combo_grad.csv', delimiter=',', index_col=0)[2:]

### ハイパーパラメータ ###
SVM_PARAMETER = pd.read_csv('../temp/svm_parameter.csv', delimiter=',', index_col=0)

for i, row in PARAM_COMBO_GRAD.iterrows():
    ### パラメータの組み合わせ ###
    ptcnt_type, ptcnt, input_col = row['ptcnt_type'], row['ptcnt'], row['X_col']
    
    ### 結果格納用df ###
    param_result = pd.read_csv('../temp/svm_parameter.csv', delimiter=',', index_col=0)
    param_result_dict = {}
    for col in param_result:
        param_result_dict[col] = param_result[col]
    for k in range(K):
        param_result_dict[f'kfold_{k}_recall'] = []
        param_result_dict[f'kfold_{k}_precision'] = []
        param_result_dict[f'kfold_{k}_f1'] = []
        param_result_dict[f'kfold_{k}_accuracy'] = []

    param_result_dict['test_recall'] = []
    param_result_dict['test_precision'] = []
    param_result_dict['test_f1'] = []
    param_result_dict['test_accuracy'] = []

    data = pd.read_csv(f'../temp/eachpt_feature_grad/{ptcnt_type}_{ptcnt}_{input_col}.csv', delimiter=',', index_col=0)
    X = data.copy()
    y = data[Y_COL]
    # 学習データとテストデータを725:310に分割する
    train_valid, test, y_train_valid, y_test = train_test_split(X, y, train_size=725, shuffle=True, stratify=y, random_state=TRAIN_TEST_SPLIT_SEED)
   
    for j, row in SVM_PARAMETER.iterrows():
        kernel, c, gamma, degree = row['kernel'], row['c'], row['gamma'], row['degree']
    
        # K-Fold
        skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=KFOLD_SHUFFLE_SEED)
        for k, (train_index, valid_index) in enumerate(skf.split(train_valid, train_valid[KFOLD_SHUFFLE_LABEL])):
            
            ### 訓練データと検証データに分ける ###
            train, valid = train_valid.iloc[train_index], train_valid.iloc[valid_index]
            
            ### 説明変数と目的変数に分ける ###
            X_col = [col for col in train_valid if 'feature_' in col]
            y_col = 'is_good_saito'
            train_X = train.copy()[X_col]
            train_y = train.copy()[y_col]
            valid_X = valid.copy()[X_col]
            valid_y = valid.copy()[y_col]
            
            ### 学習データのスケーリング(正規化, 学習データのmin, maxを検証データとテストデータに適用) ###
            scaler = MinMaxScaler()
            for col in train_X:
                train_minmax = scaler.fit(train_X[[col]])
                train_X[f'norm_{col}'] = scaler.transform(train_X[[col]])
                valid_X[f'norm_{col}'] = scaler.transform(valid_X[[col]])
                del train_X[col]
                del valid_X[col]
        
            ### モデルへの入力形式に変換する ###
            train_X = train_X.to_numpy()
            train_y = train_y.to_numpy()
            valid_X = valid_X.to_numpy()
            valid_y = valid_y.to_numpy()
        
            ### モデルを生成する ###
            model = SVC(kernel=kernel, C=c, gamma=gamma, degree=degree)
            ### モデルを学習させる ###
            result = model.fit(train_X, train_y)
            ### 検証データで精度を算出する ###
            valid_pred = model.predict(valid_X)
            ### 各指標の値(validationデータに対する) ###
            recall = recall_score(valid_y, valid_pred, average=None)[1]
            precision = precision_score(valid_y, valid_pred, average=None)[1]
            f1 = f1_score(valid_y, valid_pred, average=None)[1]
            accuracy = accuracy_score(valid_y, valid_pred)
            
            ### 結果を格納する ###
            param_result_dict[f'kfold_{k}_recall'].append(recall)
            param_result_dict[f'kfold_{k}_precision'].append(precision)
            param_result_dict[f'kfold_{k}_f1'].append(f1)
            param_result_dict[f'kfold_{k}_accuracy'].append(accuracy)
        
        ### 説明変数と目的変数に分ける ###
        X_col = [col for col in train_valid if 'feature_' in col]
        y_col = 'is_good_saito'
        train_valid_X = train_valid.copy()[X_col]
        train_valid_y = train_valid.copy()[y_col]
        test_X = test.copy()[X_col]
        test_y = test.copy()[y_col]

        ### 学習データのスケーリング(正規化, 学習データのmin, maxを検証データとテストデータに適用) ###
        scaler = MinMaxScaler()
        for col in train_valid_X:
            train_minmax = scaler.fit(train_valid_X[[col]])
            train_valid_X[f'norm_{col}'] = scaler.transform(train_valid_X[[col]])
            test_X[f'norm_{col}'] = scaler.transform(test_X[[col]])
            del train_valid_X[col]
            del test_X[col]
        
        ### モデルへの入力形式に変換する ###
        train_valid_X = train_valid_X.to_numpy()
        train_valid_y = train_valid_y.to_numpy()
        test_X = test_X.to_numpy()
        test_y = test_y.to_numpy()

        ### モデルを生成する ###
        model = SVC(kernel=kernel, C=c, gamma=gamma, degree=degree)
        ### モデルを学習させる ###
        result = model.fit(train_X, train_y)
        ### 検証データで精度を算出する ###
        test_pred = model.predict(test_X)
        ### 各指標の値(testデータに対する) ###
        recall = recall_score(test_y, test_pred, average=None)[1]
        precision = precision_score(test_y, test_pred, average=None)[1]
        f1 = f1_score(test_y, test_pred, average=None)[1]
        accuracy = accuracy_score(test_y, test_pred)
        
        ### 結果を格納する ###
        param_result_dict['test_recall'].append(recall)
        param_result_dict['test_precision'].append(precision)
        param_result_dict['test_f1'].append(f1)
        param_result_dict['test_accuracy'].append(accuracy)        
    
    param_result_aftertrain = pd.DataFrame(param_result_dict)
    param_result_aftertrain.to_csv(f'../temp/result_svm/{ptcnt_type}_{ptcnt}_{input_col}.csv')
    print(ptcnt_type, ptcnt)
    print('==================================')

2 4
2 4
2 5
2 5
2 6
2 6
2 7
2 7
2 8
2 8
2 9
2 9
2 10
2 10
2 11
2 11
2 12
2 12
2 13
2 13
2 14
2 14
2 15
2 15
2 16
2 16
2 17
2 17
2 18
2 18
2 19
2 19
2 20
2 20
2 21
2 21
2 22
2 22
2 23
2 23
2 24
2 24
2 25
2 25
2 26
2 26
2 27
2 27
2 28
2 28
2 29
2 29
2 30
2 30
2 31
2 31
2 32
2 32
2 33
2 33
2 34
2 34
2 35
2 35
2 36
2 36
2 37
2 37
2 38
2 38
2 39
2 39
2 40
2 40
2 41
2 41
2 42
2 42
2 43
2 43
2 44
2 44
2 45
2 45
2 46
2 46
2 47
2 47
2 48
2 48
2 49
2 49
2 50
2 50
2 51
2 51
2 52
2 52
2 53
2 53
2 54
2 54
2 55
2 55
2 56
2 56
2 57
2 57
2 58
2 58
2 59
2 59
2 60
2 60
2 61
2 61
2 62
2 62
2 63
2 63
2 64
2 64
2 65
2 65
2 66
2 66
2 67
2 67
2 68
2 68
2 69
2 69
2 70
2 70
2 71
2 71
2 72
2 72
2 73
2 73
2 74
2 74
2 75
2 75
2 76
2 76
2 77
2 77
2 78
2 78
2 79
2 79
2 80
2 80
2 81
2 81
2 82
2 82
2 83
2 83


##### ▶テストデータだけでやり直す

In [3]:
from sklearn.svm import SVC

### 変えないパラメータ ###
TRAIN_TEST_SPLIT_SEED = 1
KFOLD_SHUFFLE_SEED = 1
KFOLD_SHUFFLE_LABEL = 'saito_label'
Y_COL = 'is_good_saito'
K = 5

### 変えるパラメータ ###
PARAM_COMBO = pd.read_csv('../temp/param_combo.csv', delimiter=',', index_col=0)
PARAM_COMBO_GRAD = pd.read_csv('../temp/param_combo_grad.csv', delimiter=',', index_col=0)

### ハイパーパラメータ ###
SVM_PARAMETER = pd.read_csv('../temp/svm_parameter.csv', delimiter=',', index_col=0)

for i, row in PARAM_COMBO.iterrows():
    ### パラメータの組み合わせ ###
    ptcnt_type, ptcnt, input_col = row['ptcnt_type'], row['ptcnt'], row['X_col']
    
    ### 結果格納用df ###
    param_result = pd.read_csv('../temp/svm_parameter.csv', delimiter=',', index_col=0)
    param_result_dict = {}
    for col in param_result:
        param_result_dict[col] = param_result[col]
    param_result_dict['test_recall'] = []
    param_result_dict['test_precision'] = []
    param_result_dict['test_f1'] = []
    param_result_dict['test_accuracy'] = []

    data = pd.read_csv(f'../temp/eachpt_feature/{ptcnt_type}_{ptcnt}_{input_col}.csv', delimiter=',', index_col=0)
    X = data.copy()
    y = data[Y_COL]
    # 学習データとテストデータを725:310に分割する
    train_valid, test, y_train_valid, y_test = train_test_split(X, y, train_size=725, shuffle=True, stratify=y, random_state=TRAIN_TEST_SPLIT_SEED)
   
    for j, row in SVM_PARAMETER.iterrows():
        kernel, c, gamma, degree = row['kernel'], row['c'], row['gamma'], row['degree']
    
        ### 説明変数と目的変数に分ける ###
        X_col = [col for col in train_valid if 'feature_' in col]
        y_col = 'is_good_saito'
        train_valid_X = train_valid.copy()[X_col]
        train_valid_y = train_valid.copy()[y_col]
        test_X = test.copy()[X_col]
        test_y = test.copy()[y_col]

        ### 学習データのスケーリング(正規化, 学習データのmin, maxを検証データとテストデータに適用) ###
        scaler = MinMaxScaler()
        for col in train_valid_X:
            train_minmax = scaler.fit(train_valid_X[[col]])
            train_valid_X[f'norm_{col}'] = scaler.transform(train_valid_X[[col]])
            test_X[f'norm_{col}'] = scaler.transform(test_X[[col]])
            del train_valid_X[col]
            del test_X[col]
        
        ### モデルへの入力形式に変換する ###
        train_valid_X = train_valid_X.to_numpy()
        train_valid_y = train_valid_y.to_numpy()
        test_X = test_X.to_numpy()
        test_y = test_y.to_numpy()

        ### モデルを生成する ###
        model = SVC(kernel=kernel, C=c, gamma=gamma, degree=degree)
        ### モデルを学習させる ###
        result = model.fit(train_valid_X, train_valid_y)
        ### 検証データで精度を算出する ###
        test_pred = model.predict(test_X)
        ### 各指標の値(testデータに対する) ###
        recall = recall_score(test_y, test_pred, average=None)[1]
        precision = precision_score(test_y, test_pred, average=None)[1]
        f1 = f1_score(test_y, test_pred, average=None)[1]
        accuracy = accuracy_score(test_y, test_pred)
        
        ### 結果を格納する ###
        param_result_dict['test_recall'].append(recall)
        param_result_dict['test_precision'].append(precision)
        param_result_dict['test_f1'].append(f1)
        param_result_dict['test_accuracy'].append(accuracy)        
    
    param_result_aftertrain = pd.DataFrame(param_result_dict)
    param_result_aftertrain.to_csv(f'../temp/test_svm/{ptcnt_type}_{ptcnt}_{input_col}.csv')
    print(ptcnt_type, ptcnt)
    print('==================================')

c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1 3


c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\i

1 3


c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\i

...

2 48
2 48
2 48
2 49


c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


2 49
2 49
2 49
2 50


c:\Users\ispec\MyWorkSpace\python\drawing_2024\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


2 50
2 50
2 50
2 51
2 51
2 51
2 51
2 52
2 52
2 52
2 52
2 53
2 53
2 53
2 53
2 54
2 54
2 54
2 54
2 55
2 55
2 55
2 55
2 56
2 56
2 56
2 56
2 57
2 57
2 57
2 57
2 58
2 58
2 58
2 58
2 59
2 59
2 59
2 59
2 60
2 60
2 60
2 60
2 61
2 61
2 61
2 61
2 62
2 62
2 62
2 62
2 63
2 63
2 63
2 63
2 64
2 64
2 64
2 64
2 65
2 65
2 65
2 65
2 66
2 66
2 66
2 66
2 67
2 67
2 67
2 67
2 68
2 68
2 68
2 68
2 69
2 69
2 69
2 69
2 70
2 70
2 70
2 70
2 71
2 71
2 71
2 71
2 72
2 72
2 72
2 72
2 73
2 73
2 73
2 73
2 74
2 74
2 74
2 74
2 75
2 75
2 75
2 75
2 76
2 76
2 76
2 76
2 77
2 77
2 77
2 77
2 78
2 78
2 78
2 78
2 79
2 79
2 79
2 79
2 80
2 80
2 80
2 80
2 81
2 81
2 81
2 81
2 82
2 82
2 82
2 82
2 83
2 83
2 83
2 83


In [4]:
from sklearn.svm import SVC

### 変えないパラメータ ###
TRAIN_TEST_SPLIT_SEED = 1
KFOLD_SHUFFLE_SEED = 1
KFOLD_SHUFFLE_LABEL = 'saito_label'
Y_COL = 'is_good_saito'
K = 5

### 変えるパラメータ ###
PARAM_COMBO = pd.read_csv('../temp/param_combo.csv', delimiter=',', index_col=0)
PARAM_COMBO_GRAD = pd.read_csv('../temp/param_combo_grad.csv', delimiter=',', index_col=0)[2:]

### ハイパーパラメータ ###
SVM_PARAMETER = pd.read_csv('../temp/svm_parameter.csv', delimiter=',', index_col=0)

for i, row in PARAM_COMBO_GRAD.iterrows():
    ### パラメータの組み合わせ ###
    ptcnt_type, ptcnt, input_col = row['ptcnt_type'], row['ptcnt'], row['X_col']
    
    ### 結果格納用df ###
    param_result = pd.read_csv('../temp/svm_parameter.csv', delimiter=',', index_col=0)
    param_result_dict = {}
    for col in param_result:
        param_result_dict[col] = param_result[col]
    param_result_dict['test_recall'] = []
    param_result_dict['test_precision'] = []
    param_result_dict['test_f1'] = []
    param_result_dict['test_accuracy'] = []

    data = pd.read_csv(f'../temp/eachpt_feature_grad/{ptcnt_type}_{ptcnt}_{input_col}.csv', delimiter=',', index_col=0)
    X = data.copy()
    y = data[Y_COL]
    # 学習データとテストデータを725:310に分割する
    train_valid, test, y_train_valid, y_test = train_test_split(X, y, train_size=725, shuffle=True, stratify=y, random_state=TRAIN_TEST_SPLIT_SEED)
   
    for j, row in SVM_PARAMETER.iterrows():
        kernel, c, gamma, degree = row['kernel'], row['c'], row['gamma'], row['degree']
    
        ### 説明変数と目的変数に分ける ###
        X_col = [col for col in train_valid if 'feature_' in col]
        y_col = 'is_good_saito'
        train_valid_X = train_valid.copy()[X_col]
        train_valid_y = train_valid.copy()[y_col]
        test_X = test.copy()[X_col]
        test_y = test.copy()[y_col]

        ### 学習データのスケーリング(正規化, 学習データのmin, maxを検証データとテストデータに適用) ###
        scaler = MinMaxScaler()
        for col in train_valid_X:
            train_minmax = scaler.fit(train_valid_X[[col]])
            train_valid_X[f'norm_{col}'] = scaler.transform(train_valid_X[[col]])
            test_X[f'norm_{col}'] = scaler.transform(test_X[[col]])
            del train_valid_X[col]
            del test_X[col]
        
        ### モデルへの入力形式に変換する ###
        train_valid_X = train_valid_X.to_numpy()
        train_valid_y = train_valid_y.to_numpy()
        test_X = test_X.to_numpy()
        test_y = test_y.to_numpy()

        ### モデルを生成する ###
        model = SVC(kernel=kernel, C=c, gamma=gamma, degree=degree)
        ### モデルを学習させる ###
        result = model.fit(train_valid_X, train_valid_y)
        ### 検証データで精度を算出する ###
        test_pred = model.predict(test_X)
        ### 各指標の値(testデータに対する) ###
        recall = recall_score(test_y, test_pred, average=None)[1]
        precision = precision_score(test_y, test_pred, average=None)[1]
        f1 = f1_score(test_y, test_pred, average=None)[1]
        accuracy = accuracy_score(test_y, test_pred)
        
        ### 結果を格納する ###
        param_result_dict['test_recall'].append(recall)
        param_result_dict['test_precision'].append(precision)
        param_result_dict['test_f1'].append(f1)
        param_result_dict['test_accuracy'].append(accuracy)        
    
    param_result_aftertrain = pd.DataFrame(param_result_dict)
    param_result_aftertrain.to_csv(f'../temp/test_svm/{ptcnt_type}_{ptcnt}_{input_col}.csv')
    print(ptcnt_type, ptcnt)
    print('==================================')

2 5
2 5
2 6
2 6
2 7
2 7
2 8
2 8
2 9
2 9
2 10
2 10
2 11
2 11
2 12
2 12
2 13
2 13
2 14
2 14
2 15
2 15
2 16
2 16
2 17
2 17
2 18
2 18
2 19
2 19
2 20
2 20
2 21
2 21
2 22
2 22
2 23
2 23
2 24
2 24
2 25
2 25
2 26
2 26
2 27
2 27
2 28
2 28
2 29
2 29
2 30
2 30
2 31
2 31
2 32
2 32
2 33
2 33
2 34
2 34
2 35
2 35
2 36
2 36
2 37
2 37
2 38
2 38
2 39
2 39
2 40
2 40
2 41
2 41
2 42
2 42
2 43
2 43
2 44
2 44
2 45
2 45
2 46
2 46
2 47
2 47
2 48
2 48
2 49
2 49
2 50
2 50
2 51
2 51
2 52
2 52
2 53
2 53
2 54
2 54
2 55
2 55
2 56
2 56
2 57
2 57
2 58
2 58
2 59
2 59
2 60
2 60
2 61
2 61
2 62
2 62
2 63
2 63
2 64
2 64
2 65
2 65
2 66
2 66
2 67
2 67
2 68
2 68
2 69
2 69
2 70
2 70
2 71
2 71
2 72
2 72
2 73
2 73
2 74
2 74
2 75
2 75
2 76
2 76
2 77
2 77
2 78
2 78
2 79
2 79
2 80
2 80
2 81
2 81
2 82
2 82
2 83
2 83
